In [1]:
import pandas as pd
import os


In [2]:
pd.set_option("display.max_rows", None, "display.max_columns",
              None, 'display.max_colwidth', None)

In [3]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')


In [12]:
# preprocess issues and questions

df_issues = pd.read_json(os.path.join(path_dataset, 'issues_original.json'))
df_questions = pd.read_json(os.path.join(
    path_dataset, 'questions_original.json'))

stop_words = ['user', 'encounter', 'attempt', 'unable', 'try', 'seek', 'look', 'face', 'facing',
              'experienc', 'struggl', 'challenge', 'difficulty', 'issue', 'error', 'problem', 'question']

for index, row in df_issues.iterrows():
    df_issues.at[index, 'Issue_original_content'] = row['Issue_original_content'].replace(
        'Title: ', '').replace('; Content:', '')
    df_issues.at[index, 'Issue_preprocessed_content'] = row['Issue_preprocessed_content'].replace(
        'Title: ', '').replace('; Content:', '')
    gpt_summary = row['Issue_original_content_gpt_summary']
    for word in stop_words:
        gpt_summary = gpt_summary.replace(word, '')
    df_issues.at[index, 'Issue_original_content_gpt_summary'] = gpt_summary

for index, row in df_questions.iterrows():
    df_questions.at[index, 'Question_original_content'] = row['Question_original_content'].replace(
        'Title: ', '').replace('; Content:', '')
    df_questions.at[index, 'Question_preprocessed_content'] = row['Question_preprocessed_content'].replace(
        'Title: ', '').replace('; Content:', '')
    gpt_summary = row['Question_original_content_gpt_summary']
    for word in stop_words:
        gpt_summary = gpt_summary.replace(word, '')
    df_questions.at[index,
                    'Question_original_content_gpt_summary'] = gpt_summary

df_issues['Challenge_link'] = df_issues['Issue_link']
df_issues['Challenge_original_content'] = df_issues['Issue_original_content']
df_issues['Challenge_original_content_gpt_summary'] = df_issues['Issue_original_content_gpt_summary']
df_issues['Challenge_preprocessed_content'] = df_issues['Issue_preprocessed_content']
df_issues['Challenge_creation_time'] = df_issues['Issue_creation_time']
df_issues['Challenge_comment_count'] = df_issues['Issue_comment_count']
df_issues['Challenge_score'] = df_issues['Issue_upvote_count'] - df_issues['Issue_downvote_count']

df_questions['Challenge_link'] = df_questions['Question_link']
df_questions['Challenge_original_content'] = df_questions['Question_original_content']
df_questions['Challenge_original_content_gpt_summary'] = df_questions['Question_original_content_gpt_summary']
df_questions['Challenge_preprocessed_content'] = df_questions['Question_preprocessed_content']
df_questions['Challenge_creation_time'] = df_questions['Question_creation_time']
df_questions['Challenge_comment_count'] = df_questions['Question_comment_count']
df_questions['Challenge_score'] = df_questions['Question_score']

df_questions['Solution_original_content'] = df_questions['Answer_original_content']
df_questions['Solution_original_content_gpt_summary'] = df_questions['Answer_original_content_gpt_summary']
df_questions['Solution_preprocessed_content'] = df_questions['Answer_preprocessed_content']

del df_issues['Issue_title']
del df_issues['Issue_body']
del df_issues['Issue_link']
del df_issues['Issue_creation_time']
del df_issues['Issue_comment_count']
del df_issues['Issue_upvote_count']
del df_issues['Issue_downvote_count']
del df_issues['Issue_original_content']
del df_issues['Issue_original_content_gpt_summary']
del df_issues['Issue_preprocessed_content']

del df_questions['Question_title']
del df_questions['Question_body']
del df_questions['Question_link']
del df_questions['Question_creation_time']
del df_questions['Question_comment_count']
del df_questions['Question_score']
del df_questions['Question_original_content']
del df_questions['Question_original_content_gpt_summary']
del df_questions['Question_preprocessed_content']

del df_questions['Answer_original_content']
del df_questions['Answer_original_content_gpt_summary']
del df_questions['Answer_preprocessed_content']

df_all = pd.concat([df_issues, df_questions], ignore_index=True)
df_all.to_json(os.path.join(path_dataset, 'all_original.json'),
               indent=4, orient='records')

In [13]:
df_all = pd.read_json(os.path.join(path_dataset, 'all_original.json'))

# remove issues with uninformed content
for index, row in df_all.iterrows():
    if len(row['Challenge_original_content'].split()) < 4 or len(row['Challenge_original_content']) < 20:
        print(row['Challenge_original_content'])
        df_all.drop(index, inplace=True)
    elif row['Solution_original_content'] and (len(row['Solution_original_content'].split()) < 6 or len(row['Solution_original_content']) < 30):
        print(row['Solution_original_content'])
        df_all.drop(index, inplace=True)

df_all.to_json(os.path.join(path_dataset, 'all_filtered.json'),
               indent=4, orient='records')

error loading
fix import issue
logger none
i solved it, thank you.
azurebug1 point
you can try https://docs.microsoft.com/en-us/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments
this is now solved. thanks!
try this in postman.
answered on github
yes that looks correct!
thank you! all fixed.


In [ ]:
df_all = pd.read_json(os.path.join(path_dataset, 'all_topics.json'))
df_all['Challenge_creation_date'] = ''
# dates = []

for index, row in df_all.iterrows():
    if 'Git' in row['Platform']:
        date = pd.to_datetime(row['Issue_creation_time'])
    else:
        date = pd.to_datetime(row['Question_creation_time'])
    df_all.at[index, 'Challenge_creation_date'] = date.date()
    # dates.append(date.date())

# min(dates), max(dates)

In [ ]:
df_challenge = df_all.groupby(
    'Challenge_topic').size().reset_index(name='Count')
df_challenge
